<a href="https://colab.research.google.com/github/pankaj-inbetween/search-in-video/blob/main/Search_In_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pysrt langchain_astradb langchain_openai python-dotenv
import pysrt
import datetime
from langchain_astradb import AstraDBVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd

In [18]:
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass("OPENAI_API_KEY = ")
os.environ['ASTRA_DB_API_ENDPOINT'] = getpass("ASTRA_DB_API_ENDPOINT = ")
os.environ['ASTRA_DB_APPLICATION_TOKEN'] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

OPENAI_API_KEY = ··········
ASTRA_DB_API_ENDPOINT = ··········
ASTRA_DB_APPLICATION_TOKEN = ··········


In [15]:
load_dotenv()

False

In [38]:
subs = pysrt.open('sub.srt')
youtube_link = "https://www.youtube.com/watch?v=haDg_l9jjxA"
#f = open("sub.srt", "r")
#print(f.read())

whatever organic compounds are there we will isolate them now biotechnology application in agriculture there are various uh you know applications are there see with increasing population like we have read in the chapter number two of zoology which is reproductive Health there was a major issue and that major issue was related to the population increasing you know population in millions it was difficult to feed those number of individuals so what we did we tried to increase the agriculture right by that only we can provide them food so we started using some chemicals we started using some chemicals like insecticide and pesticides and we when we started using those insecticide and pesticide what happened production increased and because of that insecticide and pesticide they had some residual effects also though this lead to the Green Revolution also which later we realized


In [40]:
def split_by_timewindow(subs, window_in_minutes):
    docs = []
    t = ''
    split_time = datetime.time(hour=0,minute=window_in_minutes, second=0)
    for s in subs:
        if s.start.to_time() < split_time:
            t = t + ' ' + s.text
        else:
            start_time =  (split_time.hour*3600) + (split_time.minute*60) + (split_time.second) - 60
            docs.append(Document(page_content=t,metadata={"link":f"{youtube_link}&t={start_time}s", "start_time":start_time}))
            t = s.text
            split_time = (datetime.datetime(1, 1, 1,split_time.hour,split_time.minute, split_time.second) +  datetime.timedelta(minutes=window_in_minutes)).time()
    print(t)
    return docs

In [42]:
t=split_by_timewindow(subs, 1)
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="test",
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
    api_endpoint=os.environ["ASTRA_DB_API_ENDPOINT"],
)

In [46]:
print(t)
vstore.add_documents(t)

[]


[]

In [47]:
result = vstore.similarity_search_with_relevance_scores("bio technology application in agriculture", k=5)

data = { "subtitle": [], "link": [], "start_time": [], "score": []}
for r in result:
    data['subtitle'].append(r[0].page_content)
    data['link'].append(r[0].metadata['link'])
    data['score'].append(r[1])
    data['start_time'].append(r[0].metadata['start_time'])
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)
df


,subtitle,link,start_time,score
